In [ ]:
import akshare as ak
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
stock_list = stock_zh_a_spot_em_df["代码"]
a = len(stock_list)
a

## 获得动量因子

接口: stock_institute_hold_detail

目标地址: http://vip.stock.finance.sina.com.cn/q/go.php/vComStockHold/kind/jgcg/index.phtml

描述: 获取新浪财经-机构持股-机构持股详情

限量: 单次获取新浪财经-机构持股-机构持股详情所有数据

输入参数

名称类型必选描述stockstrYstock="300003"; 股票代码quarterstrYquarter="20201"; 从 2005 年开始, {"一季报":1, "中报":2 "三季报":3 "年报":4}, e.g., "20191", 其中的 1 表示一季报; "20193", 其中的 3 表示三季报;

输出参数

名称类型默认显示描述持股机构类型strY-持股机构代码strY-持股机构简称floatY-持股机构全称floatY-持股数floatY注意单位: 万股最新持股数floatY注意单位: 万股持股比例floatY注意单位: %最新持股比例floatY注意单位: %占流通股比例floatY注意单位: %最新占流通股比例floatY注意单位: %持股比例增幅floatY注意单位: %占流通股比例增幅floatY注意单位: %

In [ ]:
import time
import sys
import pandas as pd
import numpy as np

def momentum_acquire(stock, quarter):  
    try:        
        stock_institute_hold_detail_df = ak.stock_institute_hold_detail(stock=stock, quarter=quarter)
        if stock_institute_hold_detail_df is None:
            return(np.nan,np.nan,np.nan,np.nan)
        else:
            stock_institute_hold_detail_df_fund = stock_institute_hold_detail_df.loc[stock_institute_hold_detail_df["持股机构类型"]== "基金"]

            if stock_institute_hold_detail_df_fund.empty == True: 
                return(np.nan,np.nan,np.nan,np.nan)
            else:
                # 获取基金类型
                stock_institute_hold_detail_df_fund["Type"] = np.nan

                if len(stock_institute_hold_detail_df_fund) == 1:
                    words = list(list(stock_institute_hold_detail_df_fund["持股机构简称"])[0])
                    share_holding_tem = float(list(stock_institute_hold_detail_df_fund["持股比例"])[0])
                    stock_institute_hold_detail_df_fund["持股比例"] = [share_holding_tem]
                    delta_share_holding_tem = float(list(stock_institute_hold_detail_df_fund["持股比例增幅"])[0])
                    stock_institute_hold_detail_df_fund["持股比例增幅"] = [delta_share_holding_tem]
                    if "增" in words and "强" in words:
                        stock_institute_hold_detail_df_fund["Type"] = ["Active_Fund"]
                    else:
                        if "指" in words and "数" in words:
                            stock_institute_hold_detail_df_fund["Type"] = ["Passive_Fund"]
                        else:
                            if "E" in words and "T" in words and "F" in words:
                                stock_institute_hold_detail_df_fund["Type"] = ["Passive_Fund"]
                            else:
                                stock_institute_hold_detail_df_fund["Type"] = ["Active_Fund"]
                else:
                    stock_type = []
                    stock_sharing = []
                    delta_stock_sharing = []
                    for i in range(0,len(stock_institute_hold_detail_df_fund)):
                        words = list(list(stock_institute_hold_detail_df_fund["持股机构简称"])[i])
                        stock_sharing.append(float(list(stock_institute_hold_detail_df_fund["持股比例"])[i]))
                        delta_stock_sharing.append( float(list(stock_institute_hold_detail_df_fund["持股比例增幅"])[i]))
                        if "增" in words and "强" in words:
                            stock_type.append("Active_Fund")
                        else:
                            if "指" in words and "数" in words:
                                stock_type.append("Passive_Fund")
                            else:
                                if "E" in words and "T" in words and "F" in words:
                                    stock_type.append("Passive_Fund")
                                else:
                                    stock_type.append("Active_Fund")
                    stock_institute_hold_detail_df_fund["Type"] = stock_type
                    stock_institute_hold_detail_df_fund["持股比例"] = stock_sharing
                    stock_institute_hold_detail_df_fund["持股比例增幅"] = delta_stock_sharing

                # 被动因子获得
                stock_institute_hold_detail_df_fund_Passive_Fund = stock_institute_hold_detail_df_fund.loc[stock_institute_hold_detail_df_fund["Type"]== "Passive_Fund"]
                if stock_institute_hold_detail_df_fund_Passive_Fund.empty == True:
                    Passive_Momentum = np.nan
                    Passive_Momentum_Delta = np.nan
                else:
                    Passive_Momentum = sum(stock_institute_hold_detail_df_fund_Passive_Fund["持股比例"])
                    Passive_Momentum_Delta = sum(stock_institute_hold_detail_df_fund_Passive_Fund["持股比例增幅"])

                # 主动因子获得
                stock_institute_hold_detail_df_fund_Active_Fund = stock_institute_hold_detail_df_fund.loc[stock_institute_hold_detail_df_fund["Type"]== "Active_Fund"]
                if stock_institute_hold_detail_df_fund_Active_Fund.empty == True:
                    Active_Momentum =np.nan
                    Active_Momentum_Delta = np.nan
                else:
                    Active_Momentum = sum(stock_institute_hold_detail_df_fund_Active_Fund["持股比例"])
                    Active_Momentum_Delta = sum(stock_institute_hold_detail_df_fund_Active_Fund["持股比例增幅"])
        return (Passive_Momentum,Passive_Momentum_Delta,Active_Momentum,Active_Momentum_Delta)
        

    except Exception as e:
        error_message = str(e)
        
        # If the error message contains 'https://', exit the program
        if 'https://' in error_message:
            print(f"Encountered an error: {error_message}")
            sys.exit(1)
        
        # If the error message contains 'Max retries exceeded', return a special series
        if 'Max retries exceeded' in error_message:
            return(np.nan,np.nan,np.nan,np.nan)
        
        # For other errors, you might want to log the error and possibly re-raise it
        print(f"An unexpected error occurred: {error_message}")
        raise

In [ ]:
# 获得了 300003 在 2019 年 第一季度的 相关数据
momentum_acquire("300003","20194")

## 年份，日期与选择

### 在这里改变年份！

In [ ]:
year = 2007

#### 运行

In [ ]:
# 创建某年表格
stock_list_list = list(stock_list)
stock_list_2005 = []
for k in range(4):
    stock_list_2005 = stock_list_2005+stock_list_list

df_stocks_2005 = pd.DataFrame(columns = ["股票代码","季度","初始日期","结束日期","初始价格","结束价格","returns","PM","DPM","AM","DAM"])
df_stocks_2005["股票代码"] = stock_list_2005
df_stocks_2005

In [ ]:
# 设置某年日期
quarter_total_list = []
start_time_total_list = []
end_time_total_list = []

year = str(year)
quarter = str(1)

momentum_time_list = [year,quarter]
momentum_time = int("".join(momentum_time_list))
quarter_total_list = [momentum_time]*a

start_time_date = str("0101")
end_time_date = str("0331")
start_time_list = [year,start_time_date]
end_time_list = [year,end_time_date]
start_time = int("".join(start_time_list))
end_time = int("".join(end_time_list))

start_time_total_list = [start_time]*a
end_time_total_list = [end_time] *a

for k in range(2,5):
    quarter = str(k)

    momentum_time_list = [year,quarter]
    momentum_time = int("".join(momentum_time_list))
    quarter_tem_list = [momentum_time] * a
    quarter_total_list = quarter_total_list + quarter_tem_list
        

    if k == 2:
        start_time_date = str("0401")
        end_time_date = str("0630")
        start_time_list = [year,start_time_date]
        end_time_list = [year,end_time_date]
        start_time = int("".join(start_time_list))
        end_time = int("".join(end_time_list))
        start_time_tem_list = [start_time]*a
        end_time_tem_list = [end_time] * a
        start_time_total_list = start_time_total_list+start_time_tem_list
        end_time_total_list = end_time_total_list+end_time_tem_list

    if k == 3:
        start_time_date = str("0701")
        end_time_date = str("0930")
        start_time_list = [year,start_time_date]
        end_time_list = [year,end_time_date]
        start_time = int("".join(start_time_list))
        end_time = int("".join(end_time_list))
        start_time_tem_list = [start_time]*a
        end_time_tem_list = [end_time] * a
        start_time_total_list = start_time_total_list+start_time_tem_list
        end_time_total_list = end_time_total_list+end_time_tem_list

    if k == 4:
        start_time_date = str("1001")
        end_time_date = str("1231")
        start_time_list = [year,start_time_date]
        end_time_list = [year,end_time_date]
        start_time = int("".join(start_time_list))
        end_time = int("".join(end_time_list))
        start_time_tem_list = [start_time]*a
        end_time_tem_list = [end_time] * a
        start_time_total_list = start_time_total_list+start_time_tem_list
        end_time_total_list = end_time_total_list+end_time_tem_list

df_stocks_2005["季度"] = quarter_total_list
df_stocks_2005["初始日期"] = start_time_total_list
df_stocks_2005["结束日期"] = end_time_total_list
df_stocks_2005

## 获得价格

In [ ]:
import numpy as np
import time
import pandas as pd
import os
import akshare as ak
import json

def main_process(df_stocks_2005):
    # Define batch size and sleep time
    batch_size = 500
    sleep_time = 5

    # Define a temporary Excel file to save the data
    temp_excel_file = 'temp_storage.xlsx'

    if os.path.exists(temp_excel_file):
        df_stocks_2005 = pd.read_excel(temp_excel_file)
    else:
        if 'Processed' not in df_stocks_2005.columns:
            df_stocks_2005['Processed'] = False

    total_iterations = len(df_stocks_2005)
    resume_batch = (df_stocks_2005['Processed'] == False).idxmax() // batch_size
    print(f"Resuming from batch {resume_batch + 1}/{total_iterations // batch_size + 1}")

    def padding_stock_code(original_stock_code):
        return str(original_stock_code).zfill(6)

    def stock_vol(df):
        return np.sqrt(np.var(list(df["收盘"]/df["开盘"] - 1)))

    for k in range(resume_batch * batch_size, total_iterations, batch_size):
        df_stocks_2005.loc[df_stocks_2005['股票代码'] == 301489, 'Processed'] = True

        for i in range(k, min(k + batch_size, total_iterations)):
            if not df_stocks_2005.loc[i, 'Processed']:
                stock_index = padding_stock_code(df_stocks_2005.loc[i, "股票代码"])
                begin_date = df_stocks_2005.loc[i, "初始日期"]
                end_date = df_stocks_2005.loc[i, "结束日期"]
                print(stock_index,i,begin_date)

                max_retries = 10
                for attempt in range(max_retries):
                    try:
                        stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=stock_index, period="daily", start_date=begin_date, end_date=end_date, adjust="hfq")
                        break
                    except json.JSONDecodeError:
                        if attempt == max_retries - 1:
                            print(f"Max retries reached for stock {stock_index}. Moving to the next stock.")
                            continue

                if stock_zh_a_hist_df.empty or len(stock_zh_a_hist_df) <= 1:
                    start_price = np.nan
                    end_price = np.nan
                else:
                    start_price = stock_zh_a_hist_df.iloc[0]["开盘"]
                    end_price = stock_zh_a_hist_df.iloc[-1]["收盘"]

                if not stock_zh_a_hist_df.empty and len(stock_zh_a_hist_df) > 1:
                    df_stocks_2005.loc[i, "VOL"] = stock_vol(stock_zh_a_hist_df)
                else:
                    df_stocks_2005.loc[i, "VOL"] = np.nan
                
                df_stocks_2005.loc[i, ["初始价格", "结束价格", 'Processed']] = [start_price, end_price, True]

        df_stocks_2005.to_excel(temp_excel_file, index=False)
        print(f"Batch {k // batch_size + 1}/{total_iterations // batch_size + 1} has been saved.")
        if k + batch_size < total_iterations:
            time.sleep(sleep_time)

    return df_stocks_2005

# Self-restart mechanism
max_restart_attempts = 5
for attempt in range(max_restart_attempts):
    try:
        df_stocks_2005 = main_process(df_stocks_2005)
        break
    except Exception as e:
        print(f"Error encountered: {e}. Restarting process (Attempt {attempt + 1}/{max_restart_attempts})...")

df_stocks_2005

#### 检查故障

In [ ]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000938", period="daily", start_date=20081001, end_date=20081231, adjust="hfq")
print(len(stock_zh_a_hist_df))
stock_zh_a_hist_df

## FF5 因子

根据三个月 进行平均

In [ ]:
ff5 = pd.read_excel("./FF5.xlsx")
ff5 = ff5.dropna().reset_index()
ff5 = ff5.drop("index",axis = 1)
ff5

In [ ]:
import pandas as pd

# Define the quarter labels
quarters = [20051, 20052, 20053, 20054] + [year*10 + quarter for year in range(2006, 2023) for quarter in range(1, 5)]

# Replace with your long list of factor data (replace with your actual data)
rf_list = ff5["Rf"] # Rf factor data
mkt_list = ff5["MKT"] # MKT factor data
smb_list = ff5["SMB"]  # SMB factor data
hml_list = ff5["HML"]  # HML factor data
cma_list = ff5["CMA"]  # CMA factor data
rmw_list = ff5["RMW"]  # RMW factor data

# Create a dictionary mapping quarters to factors
data_dict = {quarter: {'Rf': rf, 'MKT': mkt, 'SMB': smb, 'HML': hml, 'CMA': cma, 'RMW': rmw} 
             for quarter, rf, mkt, smb, hml, cma, rmw in zip(quarters, rf_list, mkt_list, smb_list, hml_list, cma_list, rmw_list)}

# Create new columns in the existing dataframe for each factor, setting the values based on the "季度" column
for factor in ['Rf', 'MKT', 'SMB', 'HML', 'CMA', 'RMW']:
    df_stocks_2005[factor] = df_stocks_2005['季度'].map(lambda x: data_dict.get(x, {}).get(factor))

# Display the updated dataframe
df_stocks_2005

### 数据处理

In [ ]:
# dropna 价格

df_stocks_2005_1 = df_stocks_2005.dropna(subset=['初始价格','结束价格'])
df_stocks_2005_2 = df_stocks_2005_1.loc[df_stocks_2005_1["初始价格"] != 0.0]
df_stocks_2005_2["returns"] = (df_stocks_2005_2["结束价格"]/df_stocks_2005_2["初始价格"]) - 1
df_stocks_2005_2 = df_stocks_2005_2.reset_index()
df_stocks_2005_2 = df_stocks_2005_2.drop('index',axis=1)
df_stocks_2005_2 = df_stocks_2005_2.drop('Processed',axis = 1)
df_stocks_2005_2

### 动量获取

In [ ]:
import pandas as pd
import numpy as np
import time
import os

# Define batch size and sleep time
batch_size = 500  # Adjust as necessary
sleep_time = 3 # Adjust as necessary (time in seconds)

# Define a temporary Excel file to save the data
temp_excel_file = 'temp_storage_momentum.xlsx'

# Load the existing data if the temporary file exists, otherwise use the original DataFrame
if os.path.exists(temp_excel_file):
    df_stocks_2005_2 = pd.read_excel(temp_excel_file)
else:
    # Here, df_stocks_2005_2 should be defined before this script
    if 'Processed' not in df_stocks_2005_2.columns:
        df_stocks_2005_2['Processed'] = False

# Get the total number of iterations
total_iterations = len(df_stocks_2005_2)

# Find the batch number to resume from
resume_batch = (df_stocks_2005_2['Processed'] == False).idxmax() // batch_size
print(f"Resuming from batch {resume_batch}")

def padding_stock_code(original_stock_code):
    original_stock_code = str(original_stock_code)
    number_padding_zeros = 6 - len(original_stock_code)
    return str(number_padding_zeros * '0' + original_stock_code)

# Loop through the stocks in batches starting from the resume batch
for k in range(resume_batch * batch_size, total_iterations, batch_size):
    
    # Process the stocks in the current batch
    for i in range(k, min(k + batch_size, total_iterations)):
        # Check if the row has already been processed
        if not df_stocks_2005_2.at[i, 'Processed']:
            try:  # Start of try block to catch errors
                stock_index = df_stocks_2005_2.at[i, "股票代码"]
                stock_index = padding_stock_code(stock_index)
                momentum_time = df_stocks_2005_2.at[i, "季度"]
                print(stock_index, momentum_time, i)
                momentum_acquire_list = momentum_acquire(stock=stock_index, quarter=momentum_time)
                df_stocks_2005_2.at[i, "PM"] = momentum_acquire_list[0]
                df_stocks_2005_2.at[i, "DPM"] = momentum_acquire_list[1]
                df_stocks_2005_2.at[i, "AM"] = momentum_acquire_list[2]
                df_stocks_2005_2.at[i, "DAM"] = momentum_acquire_list[3]
                df_stocks_2005_2.at[i, 'Processed'] = True
            except Exception as e:  # Catch any exception
                if str(e) == "'data'":  # Check if the error message matches the expected error
                    print(f"An unexpected error occurred at index {i}; 'data'. Passing...")
                    df_stocks_2005_2.at[i, "PM"] = np.nan
                    df_stocks_2005_2.at[i, "DPM"] = np.nan
                    df_stocks_2005_2.at[i, "AM"] = np.nan
                    df_stocks_2005_2.at[i, "DAM"] = np.nan
                    df_stocks_2005_2.at[i, 'Processed'] = True
                    pass  # Pass the error and continue with the next iteration
                else:  # If the error is different, you might want to raise it to see what happened
                    raise
    
    # Save the updated DataFrame to the temporary Excel file
    df_stocks_2005_2.to_excel(temp_excel_file, index=False)

    
    # Sleep for the defined time before processing the next batch
    if k + batch_size < total_iterations:
        print(f"Batch completed. Sleeping for {sleep_time} seconds before starting next batch...")
        time.sleep(sleep_time)

# Display the updated dataframe
df_stocks_2005_2

#### 检查故障

In [ ]:
stock_institute_hold_detail_df = ak.stock_institute_hold_detail(stock='689009', quarter='20221')
stock_institute_hold_detail_df

In [ ]:
momentum_acquire(stock='600300',quarter=20054)

## 保存

In [ ]:
stock_list_1 = df_stocks_2005_2["股票代码"]

def padding_stock_code(orignial_stock_code):
    original_stock_code = str(orignial_stock_code)
    number_padding_zeros = 6 - len(original_stock_code)
    return str(number_padding_zeros * '0' + original_stock_code)

stock_list_new = [padding_stock_code(stock) for stock in stock_list_1]
df_stocks_2005_2["股票代码"] = stock_list_new
df_stocks_2005_2

In [ ]:
df_stocks_2005_2 = df_stocks_2005_2.drop('Processed',axis = 1)
df_stocks_2005_2.to_excel('./'+f'{year}'+'_test.xlsx')

# Delete temporary files if they exist
temp_files = ['temp_storage.xlsx', 'temp_storage_momentum.xlsx']
for temp_file in temp_files:
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"Deleted temporary file: {temp_file}")
    else:
        print(f"Temporary file not found: {temp_file}")
 
if os.path.exists('./'+f'{year}'+'_test.xlsx'):
    print("Succefully Saved "+f'{year}'+".file")
else:
    print("Save error",f'{year}')